# Surface Mining Detection

### Load packages

In [1]:
# Import functions for this notebook
from Surface_mining_screening import (
    calculate_area_per_pixel,
    load_vector_file,
    process_data,
    calculate_vegetation_loss,
    plot_possible_mining,
    plot_vegetationloss_mining,
)

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


### Get Info from User

Information and data collected here will be used for labelling the final produce among others. 

In [2]:
district_name = input("District Shapefile Name : ")

District Shapefile Name :  Bindura


### Analysis parameters

The parameters are:

* `start_year`: The year to start the analysis
* `end_year`: The year to end the analysis
* `product` : whether oto use Sentinel-2 or sentinel-1 for the analysis, acceptable values are `'s2'` or `'s1'`
* `threhold`: The threshold to use for indentifying vegetaton loss, this can either be a float e.g. `-0.15`, or `'otsu'`, in which case the threshold will be automatically identified using the otsu method.
* `vector_file`: The file path and name of a vector file containing the extent. Can be a `.kml` file, or a `.shp` file. Make sure the other files such as .prj, .shx .dbf are also available in the same folder path as that of the `.shp` file

In [3]:
start_year = 2018
end_year = 2022
product= 's2'
threshold = -0.1
vector_file = f"data/{district_name}.shp"

### View the area of interest defined by the vector file

In [4]:
gdf, geom = load_vector_file(vector_file)

Label(value='')

Map(center=[-17.23360957999995, 31.312503367500057], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Load and process data

This analysis uses two datasets:

* Sentinel-2 geomedians or Sentinel-1 backscatter for identifying vegetation loss
* Water Observations from Space (WOfS) for identifying water

If an area has experienced both vegetation loss and water presence, it may be related to surface mining activity. 

In [ ]:
sentinel, wofs = process_data(gdf, geom, start_year, end_year, product=product)

## Calculation of Vegetation Loss

In [ ]:
veg_loss_annual, veg_loss_all = calculate_vegetation_loss(sentinel, threshold=threshold, product=product)

## Determine possible mining areas

`Assumption`: When water (WOfS) and vegetation loss are coincident, the area is classified as high-potential mining site.

In [ ]:
vegetation_loss_mining = plot_possible_mining(sentinel, veg_loss_all, wofs, product=product)

## Examine vegetation loss from possible mining each year

The final function compares the vegetation loss from possible mining to all vegetation loss each year, and builds a spatial plot showing the vegetation loss from possible mining each year.

In [ ]:
plot_vegetationloss_mining(sentinel, veg_loss_annual, vegetation_loss_mining, product=product)

### Export the Analysis Result

In [ ]:
from datacube.utils.cog import write_cog
write_cog(vegetation_loss_mining, 
          fname=f"{district_name}.tif",
          overwrite=True)